In [152]:
# Import packages
import numpy as np
curDay = 11

# Download input file
from download_inputs import download_inputs

# download_inputs(curDay)

In [153]:
# Load input into workspace
filename = f'inputs\\day{curDay}.txt'
# curIn = np.loadtxt(filename)

inputFile = open(filename)
inputs = []
inputs = [s for s in inputFile.read().splitlines()]

print('Done loading input')

Done loading input


In [154]:
# Load test-input into workspace
filenameTest = f'inputs\\day{curDay}test.txt'

inputFileTest = open(filenameTest)
inputsTest = []
inputsTest = [s for s in inputFileTest.read().splitlines()]

print('Done loading test-input') 

Done loading test-input


In [184]:
curInput = inputs
# curInput = inputsTest

In [185]:
class Monkey:
    def __init__(self,MonkeyNum,items,op,test,ifTrue,ifFalse):
        self.MonkeyNum = -1
        self.items = items
        self.op = op
        self.test = test
        self.ifTrue = ifTrue
        self.ifFalse = ifFalse 
        self.inspCount = 0

    def getItems(self):
        for i in range(len(self.items)):
            print(self.items[i],end=',')
    def evalFunc(self,old):
        return eval(self.op)

# testMonkey = Monkey([3,6],'','','','')
# testMonkey.getItems()

allMonkeys = []
for curI in curInput:
    # print(curI)
    # print(curI.split('Monkey '))
    if (curI[0:2] == 'Mo'):
        # print(curI.split('Monkey ')[1][:-1])
        newMonkey = Monkey(int(curI.split('Monkey ')[1][:-1]),[],'','','','')
        # curMonkeyInputs = [int(curI.split('Monkey ')[1][:-1])]
    if (curI[0:5] == '  Sta'):
        curItems = []
        thisIn = curI.split('  Starting items: ')[1]
        for newIt in thisIn.split(', '):
            
            curItems = np.append(curItems,int(newIt))
        # print(curItems)
        newMonkey.items = curItems 
        # newMonkey.getItems()
    if (curI[0:5] == '  Ope'):
        # print(curI.split('  Operation: new = ')[1])
        curFuncStr = curI.split('  Operation: new = ')[1]
        # def newFunc(old):
        #     return eval(curFuncStr)
        # newMonkey.op = newFunc 
        # newMonkey.op = lambda old: eval(curFuncStr)
        newMonkey.op = curFuncStr
        # print(newMonkey.op(2))
    if (curI[0:5] == '  Tes'):
        curDivNum = int(curI.split('  Test: divisible by ')[1])
        newMonkey.test = curDivNum
    if (curI[0:8] == '    If t'):
        newMonkey.ifTrue = int(curI.split('    If true: throw to monkey ')[1])
    if (curI[0:8] == '    If f'):
        newMonkey.ifFalse = int(curI.split('    If false: throw to monkey ')[1])
    if (curI == ''):
        allMonkeys = np.append(allMonkeys,newMonkey)

allMonkeys = np.append(allMonkeys,newMonkey)        




In [186]:
# iniMonkeys = allMonkeys.copy()
import copy 
iniMonkeys = copy.deepcopy(allMonkeys)
for mon in iniMonkeys:
    curItems = mon.items
    print(curItems)

[57.]
[58. 93. 88. 81. 72. 73. 65.]
[65. 95.]
[58. 80. 81. 83.]
[58. 89. 90. 96. 55.]
[66. 73. 87. 58. 62. 67.]
[85. 55. 89.]
[73. 80. 54. 94. 90. 52. 69. 58.]


In [187]:
def runRound(curAllMonkeys):
    for iM,mon in enumerate(curAllMonkeys):
        curItems = mon.items
        # print(curItems)
        for it in curItems:
            # print('Insp items ',it)
            # print('New Worry: ',mon.evalFunc(it))
            # print('New Worry post div: ',np.floor(mon.evalFunc(it)/3))
            curItemVal = np.floor(mon.evalFunc(it)/3) 

            curCheck = (curItemVal % mon.test) == 0
            # print('Check: ',curCheck)
            if curCheck:
                toThrowTo = mon.ifTrue
            else:
                toThrowTo = mon.ifFalse 
            # print('Throw to: ',toThrowTo)
            curAllMonkeys[toThrowTo].items = np.append(curAllMonkeys[toThrowTo].items,curItemVal)
            # Increase inspection count
            curAllMonkeys[iM].inspCount += 1
        # Empty items
        curAllMonkeys[iM].items = []
        # print('')
    return curAllMonkeys

In [202]:
# Copy the original input
allMonkeys = copy.deepcopy(iniMonkeys)
# Go through 20 round
for roundNum in range(20):
    # Apply the round
    allMonkeys = runRound(allMonkeys)

    # print('After round ', roundNum+1)
    # for mon in allMonkeys:
    #     curItems = mon.items
    #     print(curItems)

In [203]:
# Check inspection-counts
allInsp = np.array([])
for mon in allMonkeys:
    allInsp = np.append(allInsp,mon.inspCount)
# Sort the list
allInsp.sort()
# And take the product of the last two items    
p1 = allInsp[-2] * allInsp[-1]
print(int(p1))

121450


# Part 2

In [199]:
# Get greatest common divisor of all tests
comDiv = 1
for mon in allMonkeys:
    comDiv = comDiv * mon.test
print(comDiv)


9699690


In [195]:
def runRound2(curAllMonkeys):
    for iM,mon in enumerate(curAllMonkeys):
        curItems = mon.items
        # print(curItems)
        for it in curItems:
            # curItemVal = np.floor(mon.evalFunc(it)/3) 
            # Instead of dividing by three and rounding down, the full number is now sent on
            curItemVal = mon.evalFunc(it)
            
            # Get the modulo of the common divisor, to avoid value blowing up
            curItemVal = (curItemVal % comDiv) 

            curCheck = (curItemVal % mon.test) == 0
            # print('Check: ',curCheck)
            if curCheck:
                toThrowTo = mon.ifTrue
            else:
                toThrowTo = mon.ifFalse             

            # Throw item to the correct monkey
            curAllMonkeys[toThrowTo].items = np.append(curAllMonkeys[toThrowTo].items,curItemVal)
            # Increase inspection count
            curAllMonkeys[iM].inspCount += 1
        # Empty items
        curAllMonkeys[iM].items = []
    return curAllMonkeys

In [196]:
# Copy the monkeys from the initial inputs
allMonkeys = copy.deepcopy(iniMonkeys)

# Run 10000 round-2's 
for roundNum in range(10000):
    # Run the rounds
    allMonkeys = runRound2(allMonkeys)
    # If we are in round 10000
    if (roundNum == ( 10000 - 1)):
        print('== After round ',roundNum+1,'==')
        allInsp = np.array([])
        for iM,mon in enumerate(allMonkeys):
            print(f'Monkey {iM} inpsected items {mon.inspCount} times.')
            allInsp = np.append(allInsp,mon.inspCount)
        # Sort inspection counts
        allInsp.sort()
        # Calculate the answer
        p2 = allInsp[-2] * allInsp[-1]
print(int(p2))

== After round  10000 ==
Monkey 0 inpsected items 165766 times.
Monkey 1 inpsected items 161105 times.
Monkey 2 inpsected items 159985 times.
Monkey 3 inpsected items 9209 times.
Monkey 4 inpsected items 18375 times.
Monkey 5 inpsected items 166922 times.
Monkey 6 inpsected items 13590 times.
Monkey 7 inpsected items 169205 times.
28244037010
